## Forum 데이터베이스에서 데이터를 추출해보기

In [1]:
import pandas as pd

data_inst = pd.read_csv("forum_dataset.csv")
data_inst = data_inst.fillna('')

data_inst.head()

#print(data_inst.loc[data_inst['post_type'] == 3])






ModuleNotFoundError: No module named 'pandas'

## IMG 있는 내용같은 경우

In [10]:
# img 태그가 들어있는 내용은 전부 삭제.
mask = ~data_inst['cooked'].str.contains("img", na=False)
data_inst_filtered = data_inst.loc[mask]
#print(data_inst_filtered['cooked'].iloc[:50].values)




In [11]:
# 제품 카테고리
products = {
    'IMCU': ['W7500','W7500P','W7100A'],
    'Pre_programmed_MCU': ['W7500P-S2E'],
    'iEthernet': ['W5100S','W5500','W6100',"W5300"],
    'ioModule': ['WIZ850io','WIZ550io','W5100S-io',"W5500-io","W6100-io","WIZ820io"],
    'S2EModule': ['WIZ107SR','WIZ108SR','WIZ750SR','WIZ750SR-100','WIZ750SR-105','WIZ750SR-110','WIZ550SR','WIZ550S2E','WIZ550S2E-R1','WIZ550S2E-S1','WIZ550S2E-Z1','WIZ510SSL','WIZ500SR-RP','WIZ505SR-RP','WIZ510SR-RP'],
    'wifiModule': ['WizFi250','WizFi310','WizFi360','WizFi630','WizFi630A',"WizFi630S"],
    'WIZwiki': ['WIZwiki-W7500','WIZwiki-W7500P','WIZwiki-W7500ECO'],
    'RP2040': ['WizFi360-EVB-Pico','W5500-EVB-Pico','W5100S-EVB-Pico','WIZnet Ethernet HAT'],
    'Arduino': ['WizArduino M0 ETH', 'WizArduino MEGA ETH', 'W5500 Ethernet Shield', 'W5100S Ethernet Shield', 'W6100 Ethernet Shield','WizFi310 Shield','ioShield-A','ioShield-K','ioShield-L'],
    'Tool' : ['Configuration tool','ISP']
}


# 주어진 문자열
given_string = '<p>Hello W5100S Padmini. Sorry for the late reply.</p>\n<p>When WizFi360 is set to AP mode, if you connect to it in Station mode, the message that WizFi360 is connected should be displayed, but sometimes it is not.</p>\n<p>thank you</p>'

# 제품 검색 함수
# 제품 검색 함수
def find_most_mentioned_product(products, text):
    most_mentioned_product = None
    most_mentions = 0
    text_lower = text.lower() # 텍스트를 한 번만 소문자로 변환
    # 모든 제품 카테고리와 제품 이름을 순회
    for _, items in products.items():
        for item in items:
            # 소문자로 변환된 텍스트에서 소문자로 변환된 제품명의 등장 횟수를 구함
            count = text_lower.count(item.lower())
            if count > most_mentions:
                # 가장 많이 언급된 경우 정보 갱신
                most_mentions = count
                most_mentioned_product = item

    # 가장 많이 언급된 제품 이름 반환
    return most_mentioned_product

# 함수 사용
category_found = find_most_mentioned_product(products, given_string)

# 결과 출력
print(f"Most frequent product found: {category_found}")

Most frequent product found: WizFi360


In [12]:
# 제품별로 언급된 횟수를 저장할 딕셔너리
product_mention_counts = {}

# 문자열 배열을 순회하면서 각 문자열에 대해 가장 많이 언급된 제품을 찾고, 그 결과를 집계
for text in data_inst_filtered['cooked'].values:
    mentioned_product = find_most_mentioned_product(products, text)
    if mentioned_product:  # 언급된 제품이 있으면
        product_mention_counts[mentioned_product] = product_mention_counts.get(mentioned_product, 0) + 1

# 결과를 데이터 프레임으로 변환하고, 제품 이름을 기준으로 정렬한 다음, 출력
mention_counts_df = pd.DataFrame(list(product_mention_counts.items()), columns=['Product', 'Count'])
mention_counts_df.sort_values(by='Count', ascending=False, inplace=True)
print(mention_counts_df)


               Product  Count
4                W5500   2529
10               W5300    790
12               W7500    783
2             WizFi250    578
14            WIZ107SR    301
9   Configuration tool    285
1             WizFi360    280
6            WIZ550S2E    252
0             WIZ750SR    247
18            WizFi630    207
7                W6100    204
15            WIZ550io    189
8                  ISP    182
5               W5100S    181
11            WIZ108SR    132
13              W7100A    132
19            WIZ820io     91
3             WizFi310     80
17            WIZ550SR     65
16            WIZ850io     62
21         WIZ505SR-RP      7
22          ioShield-A      6
20   WizArduino M0 ETH      2


여기는 제품 이름별로 리뷰를 담은 코드

In [15]:
import pandas as pd

# 제품별로 언급된 텍스트들을 저장할 리스트
product_texts = []

# 문자열 배열을 순회하면서 각 문자열에 대해 가장 많이 언급된 제품을 찾고, 그 결과에 따라 리스트에 추가
for text in data_inst_filtered['cooked'].values:
    mentioned_product = find_most_mentioned_product(products, text)
    if mentioned_product:  # 언급된 제품이 있으면
        product_texts.append([mentioned_product, text])  # 언급된 제품명과 텍스트를 리스트에 추가

# 결과를 DataFrame으로 변환
mention_counts_df = pd.DataFrame(product_texts, columns=['Product', 'Text'])

# DataFrame 출력
print(mention_counts_df)

# 'Product' 열이 'W5500'인 행만 필터링하여 새 DataFrame 생성
w5500_df = mention_counts_df.loc[mention_counts_df['Product'] == 'W5500']

# 필터링된 DataFrame 출력
print(w5500_df.values)
w5500_df.sort_values(by='post_number').sort_values(by='topic_id').head(30)


       Product                                               Text
0     WIZ750SR  <p>Hello,</p>\n<p>Our automated spam filter, <...
1     WizFi360  <p>Hi Team,</p>\n<p>I’m working on wizfi360-PA...
2     WIZ750SR  <p>I checked the firmware of WIZ105SR and foun...
3     WizFi250  <p>안녕하세요.<br>\n현재 DSP (TMS320F28335)의 SPI로 Wiz...
4     WizFi360  <p>Hello Padmini. Sorry for the late reply.</p...
...        ...                                                ...
7580     W5500  <p>Hi, we have a product which uses W5500 as t...
7581     W5500  <p>Hello, I am using a Wiz5500 with a CC1310 M...
7582     W5500  <p>MCU 별 예제를 공식적으로 제공하는지는 잘 모르겠습니다.<br>\niO-Li...
7583     W5500  <p>Hi <a class="mention" href="/u/mohit_singh"...
7584     W5500  <p>Hi there,<br>\nI got the problem. Theres no...

[7585 rows x 2 columns]
[['W5500'
  '<p>답변 감사합니다.<br>\n추가로 궁금한 점이 있습니다.</p>\n<p>w5500에(tcp-server로 운용) 동일한 port에 다수의 client가 접속하는 경우 미리 w5500에 socket을 client 접속 수 만큼 고려하여 생성하면 되는지요?<br>\n(예를 들면 tcp-server so

KeyError: 'post_number'

In [14]:
data_inst.sort_values(by='post_number').sort_values(by='topic_id').head(30)


,id,user_id,topic_id,post_number,raw,cooked,created_at,updated_at,reply_to_post_number,reply_count,...,self_edits,reply_quoted,via_email,raw_email,public_version,action_code,locked_by_id,image_upload_id,outbound_message_id,qa_vote_count
4,1,-1.0,1,1,A category exclusive to members with trust lev...,<p>A category exclusive to members with trust ...,2017-05-04 08:07:02.008877,2017-05-04 08:07:02.008877,,0,...,0,False,False,,1,,,,,0
18131,2,-1.0,2,1,"Discussion about this site, its organization, ...","<p>Discussion about this site, its organizatio...",2017-05-04 08:07:03.964004,2017-05-04 08:07:03.964004,,0,...,0,False,False,,1,,,,,0
19275,3,-1.0,3,1,Private category for staff discussions. Topics...,<p>Private category for staff discussions. Top...,2017-05-04 08:07:04.22,2017-05-04 08:07:04.22,,0,...,0,False,False,,1,,,,,0
28472,5,-1.0,4,2,Edit the first post in this topic to change th...,<p>Edit the first post in this topic to change...,2017-05-04 08:07:04.775163,2017-05-04 08:07:04.775163,,0,...,0,False,False,,1,,,,,0
2977,4,-1.0,4,1,The following terms and conditions govern all ...,<p>The following terms and conditions govern a...,2017-05-04 08:07:04.433876,2017-05-09 01:57:39.829896,,0,...,0,False,False,,2,,,,,0
22415,6,-1.0,5,1,"<a name=""civilized""></a>\n\n## [This is a Civi...","<p><a name=""civilized""></a></p>\n<h2>\n<a name...",2017-05-04 08:07:04.925347,2017-05-04 08:07:04.925347,,0,...,0,False,False,,1,,,,,0
5671,7,-1.0,5,2,Edit the first post in this topic to change th...,<p>Edit the first post in this topic to change...,2017-05-04 08:07:05.091754,2017-05-04 08:07:05.091754,,0,...,0,False,False,,1,,,,,0
18221,9,-1.0,6,2,Edit the first post in this topic to change th...,<p>Edit the first post in this topic to change...,2017-05-04 08:07:05.369795,2017-05-04 08:07:05.369795,,0,...,0,False,False,,1,,,,,0
18209,8,-1.0,6,1,"<a name=""collect""></a>\n\n## [What information...","<p><a name=""collect""></a></p>\n<h2>\n<a name=""...",2017-05-04 08:07:05.246445,2017-05-04 08:07:05.246445,,0,...,0,False,False,,1,,,,,0
2336,10237,3885.0,7,3,for flair (jpg)\n\n![|55x55](upload://fAV87kzL...,"<p>for flair (jpg)</p>\n<p><img src=""https://g...",2017-05-07 12:13:32.291843,2020-12-14 04:26:13.164659,,0,...,1,False,False,,2,,,942.0,,0
